In [1]:
import pandas as pd

from mace.calculators import mace_mp
import numpy as np
import pickle as pkl

from tqdm import tqdm

/Users/hojechun/miniconda3/envs/krict/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.

In [2]:
model = mace_mp("medium")

Using Materials Project MACE for MACECalculator with /Users/hojechun/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/Users/hojechun/miniconda3/envs/krict/lib/python3.10/site-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_l

In [6]:
# Select columns that start with 'fp'
handcrafted_features = pd.read_csv("datasets/processed/parsed_dataset_normalized.csv")
fp_columns = [col for col in handcrafted_features.columns if col.startswith('num')]
# Concatenate these columns into a single feature vector column
handcrafted_features['feature_vector'] = handcrafted_features[fp_columns].apply(lambda row: np.array(row), axis=1)
print(f"Num for handcrafted features: {len(handcrafted_features['feature_vector'][0])}")
# Convert the column with dictionary-like strings to actual dictionaries

# Updated function with error handling for non-JSON compatible strings
def parse_atoms(x):
    if isinstance(x, str):
        try:
            # Replace "array" with "np.array" to interpret as numpy arrays
            formatted_str = x.replace("array", "np.array").replace("'", '"')
            # Evaluate with restricted environment allowing only np.array
            return eval(formatted_str, {"np": np})
        except (SyntaxError, NameError, ValueError) as e:
            print(f"Error decoding row: {x}\nError: {e}")
            return {}  # Return an empty dictionary if parsing fails
    return {}  # Return empty dict if x is not a string

# Apply the parsing function with error handling
handcrafted_features['atoms'] = handcrafted_features['atoms'].apply(parse_atoms)


Num for handcrafted features: 64


In [8]:
from ase import Atoms

# Initialize an empty dictionary for the new dataset
new_dataset = {}

# Iterate over each row in the DataFrame
for idx, row in tqdm(handcrafted_features.iterrows()):
    # Extract values for each field from the current row
    value_per_atom = row['value_per_atom']
    h_feat = row['feature_vector']
    atoms = row['atoms']
    
    # Create the Atoms object using the dictionary in 'atoms' column
    descriptor = np.mean(model.get_descriptors(Atoms.fromdict(atoms)), axis=0)
    
    # Add the data to the new_dataset dictionary
    new_dataset[row['id']] = {
        "id": row["id"],
        "atoms": atoms,
        "descriptor": descriptor,
        "descriptor_2": h_feat,
        "value_per_atom": value_per_atom,
        "natoms": len(Atoms.fromdict(atoms))
    }

4452it [03:35, 20.65it/s]


In [16]:
print(f"Num for handcrafted features: {len(new_dataset[list(new_dataset.keys())[0]]['descriptor'])}")

Num for handcrafted features: 256


In [17]:
pkl.dump((new_dataset,handcrafted_features['mean'][0], handcrafted_features['std'][0]), open("datasets/processed/processed_dataset.pkl", "wb"))